In [1]:
import numpy as np 
import pygame 

from formula_functions import *
from animation_functions import *
from intersection_functions import create_spiral, distance
from show_parameters_functions import *
from key_controls_functions import *
from variables import *

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def main_loop(screen_width, screen_height, FPS):
    '''
    Initializations
    '''
    
    pygame.init()

    font_small = pygame.font.Font(None, 26)

    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer
    background_surface = pygame.Surface((screen_width, screen_height))
   
    line_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    spiral_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    y_axis_intersects_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    line_intersects_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    params_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)

    
    
    layers_list = [background_surface, line_layer, 
                   spiral_layer, params_layer,
                   y_axis_intersects_layer, line_intersects_layer]
    
    
    steps_dict_constants = {'t': t_step, 'v': v_step, 'w': w_step, 'k': k_step, 'x': x_step}
    
    const_params_dict = {'t': t,'v': v, 'w': w, 'k': k, 'x': vert_line_x}
    
    var_params_dict = {'t': t_additional,
                   'v': v_additional, 
                   'w': w_additional, 
                   'k': k_additional, 
                   'x': x_additional}
    
    
    # Create background coordinate system
    create_background(background_surface, screen_width, screen_height, units, length, bg_color, font_small)
    
    x, y = x_transform(0, half_screen_width, length), y_transform(6, half_screen_height, length)

    draw_vertical_line(line_layer, half_screen_width, half_screen_height, length, half_units)
    
    draw_spiral(spiral_layer, half_screen_width, half_screen_height,length,t=t, v=v,w=w,k=k)
    
    t_nth_list = calc_y_intersects_t(const_params_dict['t'], const_params_dict['w'], const_params_dict['k'])
      
        
    # Initial drawing of the y-intersection point
    draw_dots(y_axis_intersects_layer, half_screen_width, half_screen_height, length, t_nth_list, v, w, k, 'black')
    
    
    show_parameters(params_layer, font_small, const_params_dict,
                    var_params_dict)
    
    
    # Atach layers to screen
    blit_layers(win, layers_list, bg_color)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(FPS)
        
        update_screen, update_spiral, update_line = False, False, False
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
                update_screen, update_spiral,\
                const_params_dict, var_params_dict = handle_shift_key_controls(event, update_screen, update_spiral, 
                                                                          const_params_dict, var_params_dict)


        update_screen, update_spiral, \
        update_line, var_params_dict, \
        const_params_dict = handle_key_commands(layers_list, 
                                                const_params_dict, var_params_dict, steps_dict_constants, 
                                                half_screen_width, half_screen_height, length, half_units,
                                                update_screen, update_spiral, update_line)


        if update_screen:

            curr_x = const_params_dict['x'] + var_params_dict['x']

            curr_t = const_params_dict['t'] + var_params_dict['t']
            curr_v = const_params_dict['v'] + var_params_dict['v']
            curr_w = const_params_dict['w'] + var_params_dict['w']
            curr_k = const_params_dict['k'] + var_params_dict['k']

            if update_spiral:

                draw_spiral(spiral_layer, half_screen_width, half_screen_height, length,
                                t= curr_t, 
                                v= curr_v,
                                w= curr_w,
                                k= curr_k)
            if update_line:            
                draw_vertical_line(line_layer, half_screen_width, half_screen_height,
                           length, half_units, x_axis_value=curr_x)

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(curr_t, curr_w, curr_k)

            # Draw current y intersection point
            draw_dots(y_axis_intersects_layer, half_screen_width, half_screen_height, length, t_nth_list, curr_v, curr_w, curr_k, 'black')

            # A list contains the exact t-parameters for the intersection points of the spiral with 
            t_mth_list = calc_line_intersections_t(t_nth_list, curr_x,curr_v, curr_w, curr_k, i=100)

            draw_dots(line_intersects_layer, half_screen_width, half_screen_height, length, t_mth_list, curr_v, curr_w, curr_k, 'purple')

            # Show the current states of the spiral and line parameters
            show_parameters(params_layer, font_small, const_params_dict, var_params_dict)  

            # Attache the new drawings on the screen    
            blit_layers(win, layers_list, bg_color)


            pygame.display.update()

        
    # Exit
    pygame.quit()
    

In [3]:
main_loop(screen_width, screen_height, FPS)